In [ ]:
!conda install pytorch-cpu torchvision -c pytorch
!export CMAKE_PREFIX_PATH="$(dirname $(which conda))/../"
!conda install numpy pyyaml mkl mkl-include setuptools cmake cffi typing

In [1]:
!git clone https://github.com/SeanNaren/warp-ctc.git
!cd warp-ctc
!mkdir build; cd build
!cmake ..
!make
!export CUDA_HOME="/usr/local/cuda"
!cd ../pytorch_binding
!python setup.py install
!cd ..
!cd ..

Cloning into 'warp-ctc'...
remote: Counting objects: 657, done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 657 (delta 6), reused 18 (delta 6), pack-reused 636
Receiving objects: 100% (657/657), 356.71 KiB | 0 bytes/s, done.
Resolving deltas: 100% (323/323), done.
CMake Error: The source directory "/home/s_belyaev/speech-processing-hw" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
/bin/sh: 1: cd: can't cd to ../pytorch_binding
python: can't open file 'setup.py': [Errno 2] No such file or directory
